In [1]:
import pandas as pd
import numpy as np
import re

data = pd.read_csv('thanksgiving.csv',encoding = "Latin-1")
print(data.iloc[:5])

   RespondentID Do you celebrate Thanksgiving?  \
0    4337954960                            Yes   
1    4337951949                            Yes   
2    4337935621                            Yes   
3    4337933040                            Yes   
4    4337931983                            Yes   

  What is typically the main dish at your Thanksgiving dinner?  \
0                                             Turkey             
1                                             Turkey             
2                                             Turkey             
3                                             Turkey             
4                                           Tofurkey             

  What is typically the main dish at your Thanksgiving dinner? - Other (please specify)  \
0                                                NaN                                      
1                                                NaN                                      
2                            

In [2]:
print(data.columns)

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

In [3]:
if_cele = data['Do you celebrate Thanksgiving?']
if_cele.value_counts(dropna = False)

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [4]:
data = data.loc[if_cele == 'Yes']

In [5]:
food = data['What is typically the main dish at your Thanksgiving dinner?']
food.value_counts(dropna = False)

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
NaN                         6
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [6]:
Tofurkey_eater = data.loc[data['What is typically the main dish at your Thanksgiving dinner?'] == 'Tofurkey']
Tofurkey_eater['Do you typically have gravy?']

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object

In [7]:
apple_isnull = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple'])
pumpkin_isnull = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin'])
pecan_isnull = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan'])
ate_pies = apple_isnull & pumpkin_isnull & pecan_isnull
ate_pies.value_counts(dropna = False)

False    876
True     104
dtype: int64

In [8]:
def str2int(string):
    if pd.isnull(string):
        return None
    result = string.split(' ')[0]
    result = int(result.replace('+',''))
    return result

int_age = data['Age'].apply(str2int)
data['int_age'] = int_age
data['int_age'].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

In [9]:
# the median is greater than the mean, which means that the data
# is left skewed (more older people took the test)

In [10]:
def income2int(string):
    if pd.isnull(string):
        return None
    result = string.split(' ')[0]
    if result == 'Prefer':
        return None
    result = re.sub('[$,]','',result)
    return int(result)

int_income = data['How much total combined money did all members of your HOUSEHOLD earn last year?'].apply(income2int)
data['int_income'] = int_income
data['int_income'].describe()
    

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

In [11]:
# many missing value, strongly right skewed distribution

In [12]:
under_150k = data[data['int_income'] < 150000]
under_150k['How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [13]:
over_150k = data[data['int_income'] > 150000]
over_150k['How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64

In [14]:
# people with more income travel less

In [27]:
data.pivot_table(values = 'int_age', 
                 index = ['Have you ever tried to meet up with hometown friends on Thanksgiving night?'],
                 columns = ['Have you ever attended a "Friendsgiving?"'],
                 aggfunc = np.mean)

Have you ever attended a "Friendsgiving?"                  No        Yes
Have you ever tried to meet up with hometown fr...                      
No                                                  42.283702  37.010526
Yes                                                 41.475410  33.976744

In [28]:
data.pivot_table(values = 'int_income', 
                 index = ['Have you ever tried to meet up with hometown friends on Thanksgiving night?'],
                 columns = ['Have you ever attended a "Friendsgiving?"'],
                 aggfunc = np.mean)

Have you ever attended a "Friendsgiving?"                     No           Yes
Have you ever tried to meet up with hometown fr...                            
No                                                  78914.549654  72894.736842
Yes                                                 78750.000000  66019.736842

In [ ]:
# 